# Building an ITSG-33 RAG chatbot with LangChain, Ollama, and Neo4j

In [50]:
%%sh
pip install langchain langchain-community langchain-experimental json-repair pypdf neo4j

In [1]:
import boto3, json
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.docstore.document import Document


## Connect to Ollama 

In [54]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="phi3:latest",
    base_url='http://ollama-server-2:11434',
    temperature=0,
    top_p=0,
    top_k=40
)  

llm.invoke("Tell me a joke")

'Why don\'t scientists trust atoms? Because they make up everything!\n\n\nThis light-hearted joke plays on the double meaning of "make up" – both to compose or constitute, and also to fabricate or lie. It\'s important to remember that humor can vary widely in its appropriateness depending on context and audience.'

In [55]:
llm_graph_transformer = LLMGraphTransformer(llm=llm)

## Add the controls from Security Control Catalog into the knowlege graph
#### Nodes: Control, Control Enhancement, Family, Class, Document, Profile, SuggestedPriority, FunctionalGroup
#### Relationships: PART_OF, BLONGS_TO, REFERENCES, IS_SUGGESTED_FOR, ENHANCES

In [17]:
%%time
NEO4J_URI = 'bolt://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neo4jgenai'
NEO4J_DATABASE = 'neo4j'

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

CPU times: user 7.5 ms, sys: 0 ns, total: 7.5 ms
Wall time: 2.9 s


In [24]:
%%time
catalogue_query =  """
LOAD CSV WITH HEADERS FROM 'file:///itsg33-ann4a-eng.csv'
AS row

FOREACH (i in CASE WHEN row.enhancement IS NULL OR trim(row.enhancement) = '' THEN [1] ELSE [] END |
        MERGE (ctrl:Control {id:row.family + row.controlId})
        SET ctrl.name = trim(row.name),
            ctrl.definition = trim(row.definition),
            ctrl.supplementalGuidance = trim(row.supplementalGuidance),
            ctrl.genTailorAndImplGuidanceNotes = trim(row.generalTailoringAndImplementationGuidanceNotes),
            ctrl.suggestedPBMMPlaceholderValues = trim(row.suggestedPBMMPlaceholderValues),
            ctrl.PBMMProfileSpecificNotes = trim(row.PBMMProfileSpecificNotes),
            ctrl.suggestedSecretMMPlaceholderValues = trim(row.suggestedSecretMMPlaceholderValues),
            ctrl.SecretMMProfileSpecificNotes = trim(row.SecretMMProfileSpecificNotes)
        MERGE (f:Family {name:row.family})
        MERGE (ctrl)-[:PART_OF]->(f)
        FOREACH (i in CASE WHEN COALESCE(trim(row.class), '') <> '' THEN [1] ELSE [] END |
            MERGE (c:Class {name:row.class})
            MERGE (f)-[:BELONGS_TO]->(c) 
        )        
        FOREACH (i in CASE WHEN COALESCE(trim(row.references), '') <> '' THEN [1] ELSE [] END |
            MERGE (d:Document {name: row.references})
            MERGE (ctrl)-[:REFERENCES]->(d)
        )
        FOREACH (i in CASE WHEN trim(row.suggestedForPBMMProfile) = 'X' THEN [1] ELSE [] END |
                MERGE (pp:Profile {name:'PBMM'})
                MERGE (c)-[:SUGGESTED_FOR]->(pp)
        )
        FOREACH (i in CASE WHEN trim(row.suggestedForSecretMMProfile) = 'X' THEN [1] ELSE [] END |
                MERGE (sp:Profile {name:'SecretMM'})
                MERGE (c)-[:SUGGESTED_FOR]->(sp)
        )
        FOREACH (i in CASE WHEN trim(row.itSecurityFunction)  <> '' THEN [1] ELSE [] END |
                MERGE (itsf:ITSecurity {name:'IT Security Function'})
                FOREACH (i in CASE WHEN trim(row.itSecurityFunction) = 'R' THEN [1] ELSE [] END |
                    MERGE (itsf)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.itSecurityFunction) = 'S' THEN [1] ELSE [] END |
                    MERGE (itsf)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.itOperationsGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (itog:ITOperations {name:'IT Operations Group'})
                FOREACH (i in CASE WHEN trim(row.itOperationsGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (itog)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.itOperationsGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (itog)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.itProjects)  <> '' THEN [1] ELSE [] END |
                MERGE (itp:ITProjects {name:'IT Projects'})
                FOREACH (i in CASE WHEN trim(row.itProjects) = 'R' THEN [1] ELSE [] END |
                    MERGE (itp)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.itProjects) = 'S' THEN [1] ELSE [] END |
                    MERGE (itp)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (phs:PhysicalSecurity {name:'Physical Security Group'})
                FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (phs)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (phs)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (ps:PersonnelSecurity {name:'Personnel Security Group'})
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (ps)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (ps)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.learningCenter)  <> '' THEN [1] ELSE [] END |
                MERGE (lc:LearningCenter {name:'Learning Center'})
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (lc)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (lc)-[:SUPPORTS]->(ctrl)
                )                    
        )
        FOREACH (i in CASE 
                        WHEN trim(row.suggestedPriority)  <> '' 
                            AND trim(row.suggestedPriority)  <> 'None defined' 
                        THEN [1] ELSE [] END |
                MERGE (priority:Priority {name:trim(row.suggestedPriority)})
                MERGE (ctrl)-[:HAS_SUGGESTED_PRIORITY]->(priority)
        )
)
"""

kg.query(catalogue_query)


CPU times: user 2.3 ms, sys: 254 µs, total: 2.56 ms
Wall time: 1.09 s


[]

In [25]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Document {name: STRING}
Control {id: STRING, definition: STRING, supplementalGuidance: STRING, name: STRING, genTailorAndImplGuidanceNotes: STRING, SecretMMProfileSpecificNotes: STRING, suggestedPBMMPlaceholderValues: STRING, suggestedSecretMMPlaceholderValues: STRING, PBMMProfileSpecificNotes: STRING}
Profile {name: STRING}
Family {name: STRING}
Class {name: STRING}
ITSecurity {name: STRING}
LearningCenter {name: STRING}
Priority {name: STRING}
ITOperations {name: STRING}
PersonnelSecurity {name: STRING}
ITProjects {name: STRING}
PhysicalSecurity {name: STRING}
Relationship properties:

The relationships:
(:Control)-[:PART_OF]->(:Family)
(:Control)-[:HAS_SUGGESTED_PRIORITY]->(:Priority)
(:Control)-[:REFERENCES]->(:Document)
(:Family)-[:BELONGS_TO]->(:Class)
(:ITSecurity)-[:RESPONSIBLE_FOR]->(:Control)
(:ITSecurity)-[:SUPPORTS]->(:Control)
(:LearningCenter)-[:RESPONSIBLE_FOR]->(:Control)
(:LearningCenter)-[:SUPPORTS]->(:Control)
(:ITOperations)-[:RESPONSIBLE_FOR]->(:Co

In [56]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=kg, llm=llm, verbose=True)

In [36]:
response = chain.invoke({"query": "What are the Families?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:Family) RETURN f.name AS FamilyName;
Full Context:
[{'FamilyName': 'SI'}, {'FamilyName': 'AC'}, {'FamilyName': 'AT'}, {'FamilyName': 'AU'}, {'FamilyName': 'CA'}, {'FamilyName': 'CM'}, {'FamilyName': 'CP'}, {'FamilyName': 'IA'}, {'FamilyName': 'IR'}, {'FamilyName': 'MA'}]

> Finished chain.


{'query': 'What are the Families?',
 'result': 'SI, AC, AT, AU, CA, CM, CP, IA, IR, MA.'}

In [37]:
response = chain.invoke({"query": "how many control families are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control)-[:PART_OF]->(f:Family) RETURN COUNT(DISTINCT f) AS count
Full Context:
[{'count': 17}]

> Finished chain.


{'query': 'how many control families are there?',
 'result': 'There are 17 control families.'}

In [61]:
response = chain.invoke({"query": "List all the distinct control families"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control)-[:PART_OF]->(f:Family) RETURN DISTINCT f.name;
Full Context:
[{'f.name': 'SI'}, {'f.name': 'AC'}, {'f.name': 'AT'}, {'f.name': 'AU'}, {'f.name': 'CA'}, {'f.name': 'CM'}, {'f.name': 'CP'}, {'f.name': 'IA'}, {'f.name': 'IR'}, {'f.name': 'MA'}]

> Finished chain.


{'query': 'List all the distinct control families',
 'result': 'The distinct control families are SI, AC, AT, AU, CA, CM, CP, IA, IR, and MA.'}

In [68]:
response = chain.invoke({"query": "What are the names of P2 priority controls"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control)-[:HAS_SUGGESTED_PRIORITY]->(p:Priority {name: 'P2'})
RETURN c.name;
Full Context:
[{'c.name': 'PERSONNEL SANCTIONS'}, {'c.name': 'IDENTIFICATION AND AUTHENTICATION (NON-ORGANIZATIONAL USERS)'}, {'c.name': 'SESSION LOCK'}, {'c.name': 'CONTINUOUS MONITORING'}, {'c.name': 'USER-BASED COLLABORATION AND INFORMATION SHARING'}, {'c.name': 'PREVIOUS LOGON (ACCESS) NOTIFICATION'}, {'c.name': 'VOICE OVER INTERNET PROTOCOL'}, {'c.name': 'AUDIT REDUCTION AND REPORT GENERATION'}, {'c.name': 'USE OF EXTERNAL INFORMATION SYSTEMS'}, {'c.name': 'INCIDENT MONITORING'}]

> Finished chain.


{'query': 'What are the names of P2 priority controls',
 'result': 'The names of P2 priority controls include Personnel Sanctions, Identification and Authentication (Non-Organizational Users), Session Lock, Continuous Monitoring, User-Based Collaboration and Information Sharing, Previous Logon (Access) Notification, Voice over Internet Protocol, Audit Reduction and Report Generation, Use of External Information Systems, and Incident Monitoring.'}

In [92]:
response = chain.invoke({"query": "What controls are related to privilege? you can look at the control definitions and control supplemental guidance"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control)-[:HAS_SUGGESTED_PRIORITY]->(p:Priority)
WHERE c.definition CONTAINS "privilege" OR c.supplementalGuidance CONTAINS "privilege"
RETURN c.id, c.name
Full Context:
[{'c.id': 'IA5', 'c.name': 'AUTHENTICATOR MANAGEMENT'}, {'c.id': 'AU2', 'c.name': 'AUDITABLE EVENTS'}, {'c.id': 'PL4', 'c.name': 'RULES OF BEHAVIOUR'}, {'c.id': 'PS5', 'c.name': 'PERSONNEL TRANSFER'}, {'c.id': 'AC2', 'c.name': 'ACCOUNT MANAGEMENT'}, {'c.id': 'AC5', 'c.name': 'SEPARATION OF DUTIES'}, {'c.id': 'IA2', 'c.name': 'IDENTIFICATION AND AUTHENTICATION (ORGANIZATIONAL USERS)'}, {'c.id': 'PL8', 'c.name': 'INFORMATION SECURITY ARCHITECTURE'}, {'c.id': 'AC6', 'c.name': 'LEAST PRIVILEGE'}, {'c.id': 'PS7', 'c.name': 'THIRD-PARTY PERSONNEL SECURITY'}]

> Finished chain.


{'query': 'What controls are related to privilege? you can look at the control definitions and control supplemental guidance',
 'result': 'Account Management (AC2) and Least Privilege (AC6) are controls that relate to privilege. These areas provide guidelines for managing user access rights within an organization, ensuring appropriate levels of permissions are granted.'}

In [93]:
response = chain.invoke({"query": "How many controls are IT Security responsible for?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control)-[:SUPPORTS]->(:ITSecurity) RETURN count(c)
Full Context:
[{'count(c)': 0}]

> Finished chain.


{'query': 'How many controls are IT Security responsible for?',
 'result': "I'm sorry, but without specific data on control counts by department or responsibility area, I can't provide an exact number of controls that IT Security is responsible for. If you have a list of controls and their respective departments, I could assist further with the information available."}

In [96]:
response = chain.invoke({"query": "can you list those controls for which IT Security is responsible?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control)-[:RESPONSIBLE_FOR]->(:ITSecurity) RETURN c.name
Full Context:
[]

> Finished chain.


{'query': 'can you list those controls for which IT Security is responsible?',
 'result': "I'm sorry, but without specific details on the controls and their respective responsibilities, I can't provide a list of IT security-related controls at this time. If you have more information or context to share, that would be very helpful in giving an accurate response."}

In [100]:
response = chain.invoke({"query": "Refelct on the query again, and tell me those controls that IT Security is responsible"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control)-[:RESPONSIBLE_FOR]->(itSec:ITSecurity) RETURN c;
Full Context:
[]

> Finished chain.


{'query': 'Refelct on the query again, and tell me those controls that IT Security is responsible',
 'result': "I apologize for not having specific details at hand right now, but generally speaking, IT Security teams are typically responsible for implementing various controls to ensure data protection, system integrity, and compliance with relevant regulations. These may include access control measures, encryption protocols, network security policies, and regular audits among others. For precise controls related to a specific organization's IT Security responsibilities, I would recommend consulting their internal documentation or contacting the IT department directly."}

In [106]:
response = chain.invoke({"query": "IT Security is reponsible for certain controls. Please tell me the names of those controls"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (itSec:ITSecurity)-[:RESPONSIBLE_FOR]->(control:Control) RETURN control.name
Full Context:
[{'control.name': 'PLAN OF ACTION AND MILESTONES'}, {'control.name': 'SECURITY AUTHORIZATION'}, {'control.name': 'INFORMATION SYSTEM MONITORING'}, {'control.name': 'ACCESS CONTROL DECISIONS'}, {'control.name': 'AUDITABLE EVENTS'}, {'control.name': 'SECURITY PLANNING POLICY AND PROCEDURES'}, {'control.name': 'RULES OF BEHAVIOUR'}, {'control.name': 'INCIDENT RESPONSE TESTING AND EXERCISES'}, {'control.name': 'TELECOMMUNICATIONS SERVICES'}, {'control.name': 'INFORMATION OUTPUT HANDLING AND RETENTION'}]

> Finished chain.


{'query': 'IT Security is reponsible for certain controls. Please tell me the names of those controls',
 'result': 'IT Security is responsible for several controls, including PLAN OF ACTION AND MILESTONES, SECURITY AUTHORIZATION, INFORMATION SYSTEM MONITORING, ACCESS CONTROL DECISIONS, and SECURITY PLANNING POLICY AND PROCEDURES.'}

In [107]:
response = chain.invoke({"query": "IT Security supports certain controls. Please tell me the names of those controls"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (itSec:ITSecurity)-[:SUPPORTS]->(control:Control) RETURN DISTINCT control.name
Full Context:
[{'control.name': 'SYSTEM SECURITY PLAN'}, {'control.name': 'DEVELOPMENT PROCESS, STANDARDS, AND TOOL'}, {'control.name': 'RESPONSE TO AUDIT PROCESSING FAILURES'}, {'control.name': 'SECURITY ATTRIBUTES'}, {'control.name': 'IDENTIFICATION AND AUTHENTICATION (ORGANIZATIONAL USERS)'}, {'control.name': 'DEVELOPER SECURITY TESTING'}, {'control.name': 'COLLABORATIVE COMPUTING DEVICES'}, {'control.name': 'DEVICE IDENTIFICATION AND AUTHENTICATION'}, {'control.name': 'PERSONNEL SECURITY POLICY AND PROCEDURES'}, {'control.name': 'WIRELESS ACCESS'}]

> Finished chain.


{'query': 'IT Security supports certain controls. Please tell me the names of those controls',
 'result': 'IT Security supports several controls, including SYSTEM SECURITY PLAN, DEVELOPMENT PROCESS, STANDARDS, AND TOOL; RESPONSE TO AUDIT PROCESSING FAILURES; IDENTIFICATION AND AUTHENTICATION for organizational users; DEVELOPER SECURITY TESTING; COLLABORATIVE COMPUTING DEVICES; DEVICE IDENTIFICATION AND AUTHENTICATION; PERSONNEL SECURITY POLICY AND PROCEDURES; and WIRELESS ACCESS.'}

In [118]:
response = chain.invoke({"query": "Personnel Security is responsile for certain controls. What are those controls?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (ps:PersonnelSecurity)-[:RESPONSIBLE_FOR]->(c:Control) RETURN c.name;
Full Context:
[{'c.name': 'PERSONNEL TERMINATION'}, {'c.name': 'PERSONNEL SANCTIONS'}, {'c.name': 'POSITION CATEGORIZATION'}, {'c.name': 'ACCESS AGREEMENTS'}, {'c.name': 'PERSONNEL SCREENING'}, {'c.name': 'PERSONNEL SECURITY POLICY AND PROCEDURES'}, {'c.name': 'PERSONNEL TRANSFER'}, {'c.name': 'THIRD-PARTY PERSONNEL SECURITY'}]

> Finished chain.


{'query': 'Personnel Security is responsile for certain controls. What are those controls?',
 'result': 'Personnel Security oversees various controls, including personnel termination procedures, sanctions related to personnel, position categorization, access agreements, screening processes, security policies and procedures, personnel transfer protocols, as well as third-party personnel security measures.'}

In [122]:
response = chain.invoke({"query": "What is the definition of control with id 'CA9'?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control {id:'CA9'}) RETURN c.definition
Full Context:
[{'c.definition': '(A)\tThe organization authorizes internal connections of [Assignment: organization-defined information system components or classes of components] to the information system.\n(B)\tThe organization documents, for each internal connection, the interface characteristics, security requirements, and the nature of the information communicated.'}]

> Finished chain.


{'query': "What is the definition of control with id 'CA9'?",
 'result': 'Control CA9 refers to an organizational authorization that allows for defined internal connections between system components or classes of components. This includes documenting interface characteristics, security requirements, and the nature of information communicated through each connection.'}

In [131]:
response = chain.invoke({"query": "are the any guidance on control 'AC1'?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Control {id: 'AC1'})-[:REFERENCES]->(d:Document)
WHERE d.name IS NOT NULL
RETURN c LIMIT 1;
Full Context:
[{'c': {'suggestedPBMMPlaceholderValues': '(A) (B) frequency [at a frequency no longer than annually]', 'suggestedSecretMMPlaceholderValues': '(A) (B) frequency [at a frequency no longer than annually]', 'name': 'ACCESS CONTROL POLICY AND PROCEDURES', 'definition': '(A)\tThe organization develops, documents, and disseminates to [Assignment: organization-defined personnel or roles]:\n(a)\tAn access control policy that addresses purpose, scope, roles, responsibilities, management commitment, coordination among organizational entities, and compliance; and\n(b)\tProcedures to facilitate the implementation of the access control policy and associated access controls.\n(B)\tThe organization reviews and updates the current:\n(a)\tAccess control policy [Assignment: organization-defined frequency]; and\n(b)\tAccess contr

{'query': "are the any guidance on control 'AC1'?",
 'result': 'Yes, there is guidance on control AC1 which outlines the establishment of policies and procedures for effective implementation of selected security controls. It reflects applicable legislation and standards while considering the organizational risk management strategy.'}

## Fix the Control IDs to add a '-' between the family and the control id

In [132]:
%%time
catalogue_query =  """
MATCH (c:Control)
return c.id
"""
kg.query(catalogue_query)

CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 27 ms


[{'c.id': 'SC32'},
 {'c.id': 'SC33'},
 {'c.id': 'SC34'},
 {'c.id': 'SC35'},
 {'c.id': 'SC36'},
 {'c.id': 'SC37'},
 {'c.id': 'SC38'},
 {'c.id': 'SC39'},
 {'c.id': 'SC40'},
 {'c.id': 'SC41'},
 {'c.id': 'SC42'},
 {'c.id': 'SC43'},
 {'c.id': 'SC44'},
 {'c.id': 'SC100'},
 {'c.id': 'SC101'},
 {'c.id': 'SI1'},
 {'c.id': 'SI2'},
 {'c.id': 'SI3'},
 {'c.id': 'SI4'},
 {'c.id': 'SI5'},
 {'c.id': 'SI6'},
 {'c.id': 'SI7'},
 {'c.id': 'SI8'},
 {'c.id': 'SI9'},
 {'c.id': 'SI10'},
 {'c.id': 'SI11'},
 {'c.id': 'SI12'},
 {'c.id': 'SI13'},
 {'c.id': 'SI14'},
 {'c.id': 'SI15'},
 {'c.id': 'SI16'},
 {'c.id': 'SI17'},
 {'c.id': 'AC1'},
 {'c.id': 'AC2'},
 {'c.id': 'AC3'},
 {'c.id': 'AC4'},
 {'c.id': 'AC5'},
 {'c.id': 'AC6'},
 {'c.id': 'AC7'},
 {'c.id': 'AC8'},
 {'c.id': 'AC9'},
 {'c.id': 'AC10'},
 {'c.id': 'AC11'},
 {'c.id': 'AC12'},
 {'c.id': 'AC13'},
 {'c.id': 'AC14'},
 {'c.id': 'AC15'},
 {'c.id': 'AC16'},
 {'c.id': 'AC17'},
 {'c.id': 'AC18'},
 {'c.id': 'AC19'},
 {'c.id': 'AC20'},
 {'c.id': 'AC21'},
 {'c.id':

In [136]:
%%time
catalogue_query =  """
MATCH (c:Control)
SET c.id = substring(c.id, 0, 2) + "-" + substring(c.id, 2)
"""
kg.query(catalogue_query)

CPU times: user 2.62 ms, sys: 0 ns, total: 2.62 ms
Wall time: 96.3 ms


[]

In [137]:
%%time
catalogue_query =  """
MATCH (c:Control)
return c.id
"""
kg.query(catalogue_query)

CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 13.9 ms


[{'c.id': 'SC-32'},
 {'c.id': 'SC-33'},
 {'c.id': 'SC-34'},
 {'c.id': 'SC-35'},
 {'c.id': 'SC-36'},
 {'c.id': 'SC-37'},
 {'c.id': 'SC-38'},
 {'c.id': 'SC-39'},
 {'c.id': 'SC-40'},
 {'c.id': 'SC-41'},
 {'c.id': 'SC-42'},
 {'c.id': 'SC-43'},
 {'c.id': 'SC-44'},
 {'c.id': 'SC-100'},
 {'c.id': 'SC-101'},
 {'c.id': 'SI-1'},
 {'c.id': 'SI-2'},
 {'c.id': 'SI-3'},
 {'c.id': 'SI-4'},
 {'c.id': 'SI-5'},
 {'c.id': 'SI-6'},
 {'c.id': 'SI-7'},
 {'c.id': 'SI-8'},
 {'c.id': 'SI-9'},
 {'c.id': 'SI-10'},
 {'c.id': 'SI-11'},
 {'c.id': 'SI-12'},
 {'c.id': 'SI-13'},
 {'c.id': 'SI-14'},
 {'c.id': 'SI-15'},
 {'c.id': 'SI-16'},
 {'c.id': 'SI-17'},
 {'c.id': 'AC-1'},
 {'c.id': 'AC-2'},
 {'c.id': 'AC-3'},
 {'c.id': 'AC-4'},
 {'c.id': 'AC-5'},
 {'c.id': 'AC-6'},
 {'c.id': 'AC-7'},
 {'c.id': 'AC-8'},
 {'c.id': 'AC-9'},
 {'c.id': 'AC-10'},
 {'c.id': 'AC-11'},
 {'c.id': 'AC-12'},
 {'c.id': 'AC-13'},
 {'c.id': 'AC-14'},
 {'c.id': 'AC-15'},
 {'c.id': 'AC-16'},
 {'c.id': 'AC-17'},
 {'c.id': 'AC-18'},
 {'c.id': 'AC-19

## Add the missing control enhancements

In [140]:
%%time
catalogue_query =  """
LOAD CSV WITH HEADERS FROM 'file:///itsg33-ann4a-eng.csv'
AS row

FOREACH (i in CASE WHEN COALESCE(trim(row.enhancement), '') <> '' THEN [1] ELSE [] END |
        MERGE (ce:ControlEnhancement {id:row.family + '-' + row.controlId + '-' + row.enhancement})
        SET ce.name = trim(row.name),
            ce.definition = trim(row.definition),
            ce.supplementalGuidance = trim(row.supplementalGuidance),
            ce.genTailorAndImplGuidanceNotes = trim(row.generalTailoringAndImplementationGuidanceNotes),
            ce.suggestedPBMMPlaceholderValues = trim(row.suggestedPBMMPlaceholderValues),
            ce.PBMMProfileSpecificNotes = trim(row.PBMMProfileSpecificNotes),
            ce.suggestedSecretMMPlaceholderValues = trim(row.suggestedSecretMMPlaceholderValues),
            ce.SecretMMProfileSpecificNotes = trim(row.SecretMMProfileSpecificNotes)
        MERGE (f:Family {name:row.family})
        MERGE (ce)-[:PART_OF]->(f)
        
        MERGE (control:Control {id:row.family + '-' + row.controlId})
        MERGE (ce)-[:ENHANCES]->(control)
        
        FOREACH (i in CASE WHEN COALESCE(trim(row.class), '') <> '' THEN [1] ELSE [] END |
            MERGE (c:Class {name:row.class})
            MERGE (f)-[:BELONGS_TO]->(c) 
        )        
        FOREACH (i in CASE WHEN COALESCE(trim(row.references), '') <> '' THEN [1] ELSE [] END |
            MERGE (d:Document {name: row.references})
            MERGE (ce)-[:REFERENCES]->(d)
        )
        FOREACH (i in CASE WHEN trim(row.suggestedForPBMMProfile) = 'X' THEN [1] ELSE [] END |
                MERGE (pp:Profile {name:'PBMM'})
                MERGE (c)-[:SUGGESTED_FOR]->(pp)
        )
        FOREACH (i in CASE WHEN trim(row.suggestedForSecretMMProfile) = 'X' THEN [1] ELSE [] END |
                MERGE (sp:Profile {name:'SecretMM'})
                MERGE (c)-[:SUGGESTED_FOR]->(sp)
        )
        FOREACH (i in CASE WHEN trim(row.itSecurityFunction)  <> '' THEN [1] ELSE [] END |
                MERGE (itsf:ITSecurity {name:'IT Security Function'})
                FOREACH (i in CASE WHEN trim(row.itSecurityFunction) = 'R' THEN [1] ELSE [] END |
                    MERGE (itsf)-[:RESPONSIBLE_FOR]->(ce)
                )
                FOREACH (i in CASE WHEN trim(row.itSecurityFunction) = 'S' THEN [1] ELSE [] END |
                    MERGE (itsf)-[:SUPPORTS]->(ce)
                )
        )
        FOREACH (i in CASE WHEN trim(row.itOperationsGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (itog:ITOperations {name:'IT Operations Group'})
                FOREACH (i in CASE WHEN trim(row.itOperationsGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (itog)-[:RESPONSIBLE_FOR]->(ce)
                )
                FOREACH (i in CASE WHEN trim(row.itOperationsGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (itog)-[:SUPPORTS]->(ce)
                )
        )
        FOREACH (i in CASE WHEN trim(row.itProjects)  <> '' THEN [1] ELSE [] END |
                MERGE (itp:ITProjects {name:'IT Projects'})
                FOREACH (i in CASE WHEN trim(row.itProjects) = 'R' THEN [1] ELSE [] END |
                    MERGE (itp)-[:RESPONSIBLE_FOR]->(ce)
                )
                FOREACH (i in CASE WHEN trim(row.itProjects) = 'S' THEN [1] ELSE [] END |
                    MERGE (itp)-[:SUPPORTS]->(ce)
                )
        )
        FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (phs:PhysicalSecurity {name:'Physical Security Group'})
                FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (phs)-[:RESPONSIBLE_FOR]->(ce)
                )
                FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (phs)-[:SUPPORTS]->(ce)
                )
        )
        FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (ps:PersonnelSecurity {name:'Personnel Security Group'})
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (ps)-[:RESPONSIBLE_FOR]->(ce)
                )
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (ps)-[:SUPPORTS]->(ce)
                )
        )
        FOREACH (i in CASE WHEN trim(row.learningCenter)  <> '' THEN [1] ELSE [] END |
                MERGE (lc:LearningCenter {name:'Learning Center'})
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (lc)-[:RESPONSIBLE_FOR]->(ce)
                )
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (lc)-[:SUPPORTS]->(ce)
                )                    
        )
        FOREACH (i in CASE 
                        WHEN trim(row.suggestedPriority)  <> '' 
                            AND trim(row.suggestedPriority)  <> 'None defined' 
                        THEN [1] ELSE [] END |
                MERGE (priority:Priority {name:trim(row.suggestedPriority)})
                MERGE (ctrl)-[:HAS_SUGGESTED_PRIORITY]->(priority)
        )
)
"""

kg.query(catalogue_query)


CPU times: user 2.36 ms, sys: 0 ns, total: 2.36 ms
Wall time: 1.38 s


[]

In [151]:
kg.refresh_schema()
print(kg.schema)
chain = GraphCypherQAChain.from_llm(graph=kg, llm=llm, verbose=True)

Node properties:
Document {name: STRING}
Control {supplementalGuidance: STRING, definition: STRING, id: STRING, name: STRING, genTailorAndImplGuidanceNotes: STRING, SecretMMProfileSpecificNotes: STRING, suggestedPBMMPlaceholderValues: STRING, suggestedSecretMMPlaceholderValues: STRING, PBMMProfileSpecificNotes: STRING}
Profile {name: STRING}
Family {name: STRING}
Class {name: STRING}
ITSecurity {name: STRING}
LearningCenter {name: STRING}
Priority {name: STRING}
ITOperations {name: STRING}
PersonnelSecurity {name: STRING}
ITProjects {name: STRING}
PhysicalSecurity {name: STRING}
ControlEnhancement {id: STRING, definition: STRING, supplementalGuidance: STRING, genTailorAndImplGuidanceNotes: STRING, name: STRING, suggestedSecretMMPlaceholderValues: STRING, suggestedPBMMPlaceholderValues: STRING, PBMMProfileSpecificNotes: STRING, SecretMMProfileSpecificNotes: STRING}
Relationship properties:

The relationships:
(:Control)-[:PART_OF]->(:Family)
(:Control)-[:HAS_SUGGESTED_PRIORITY]->(:Prior

In [153]:
response = chain.invoke({"query": "ControlEhancement enhances Control. Are there any for control 'AC-2'?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:ControlEnhancement)-[:ENHANCES]->(control:Control)
WHERE control.id = "AC-2"
RETURN c;
Full Context:
[{'c': {'name': 'ACCOUNT MANAGEMENT', 'definition': 'ACCOUNT MANAGEMENT | SHARED / GROUP ACCOUNT CREDENTIAL TERMINATION\nThe information system terminates shared/group account credentials when members leave the group.', 'id': 'AC-2-(10)\t'}}, {'c': {'name': 'ACCOUNT MANAGEMENT', 'definition': 'ACCOUNT MANAGEMENT | DISABLE ACCOUNTS FOR HIGH-RISK INDIVIDUALS\nThe organization disables accounts of users posing a significant risk within [Assignment: organization-defined time period] of discovery of the risk.', 'id': 'AC-2-(13)\t', 'supplementalGuidance': 'Users posing a significant risk to organizations include individuals for whom reliable evidence or intelligence indicates either the intention to use authorized access to information systems to cause harm or through whom adversaries will cause harm. Harm includes poten

{'query': "ControlEhancement enhances Control. Are there any for control 'AC-2'?",
 'result': "Yes, there are several enhancements related to the control 'AC-2'. These include ACCOUNT MANAGEMENT | AUTOMATED SYSTEM ACCOUNT MANAGEMENT which employs automated mechanisms to support the management of information system accounts. Additionally, ACCOUNT MANAGEMENT | DISABLE INACTIVE ACCOUNTS automatically disables inactive accounts after a predefined time period as defined by the organization."}

In [156]:
response = chain.invoke({"query": "ControlEnahancement enhances Control. List all of those ControlEnahancements for control 'AC-2'?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:ControlEnhancement)-[:ENHANCES]->(control:Control {id:'AC-2'}) RETURN c;
Full Context:
[{'c': {'name': 'ACCOUNT MANAGEMENT', 'definition': 'ACCOUNT MANAGEMENT | SHARED / GROUP ACCOUNT CREDENTIAL TERMINATION\nThe information system terminates shared/group account credentials when members leave the group.', 'id': 'AC-2-(10)\t'}}, {'c': {'name': 'ACCOUNT MANAGEMENT', 'definition': 'ACCOUNT MANAGEMENT | DISABLE ACCOUNTS FOR HIGH-RISK INDIVIDUALS\nThe organization disables accounts of users posing a significant risk within [Assignment: organization-defined time period] of discovery of the risk.', 'id': 'AC-2-(13)\t', 'supplementalGuidance': 'Users posing a significant risk to organizations include individuals for whom reliable evidence or intelligence indicates either the intention to use authorized access to information systems to cause harm or through whom adversaries will cause harm. Harm includes potential adverse i

{'query': "ControlEnahancement enhances Control. List all of those ControlEnahancements for control 'AC-2'?",
 'result': "The following Account Management Control Enhancements are associated with the control 'AC-2':\n\n1. AC-2-(1): Automated System Account Management - The organization employs automated mechanisms to support the management of information system accounts.\n\n2. AC-2-(4): Automated Audit Actions - The information system automatically audits account creation, modification, enabling, disabling, and removal actions, and notifies designated personnel or roles.\n\n3. AC-2-(7)(a): Role-Based Schemes for Privileged User Accounts - The organization establishes and administers privileged user accounts in accordance with a role-based access scheme that organizes allowed information system access and privileges into roles.\n\n4. AC-2-(9): Restrictions on Use of Shared/Group Accounts - Only permits the use of shared/group accounts that meet organization-defined conditions for establ